In [ ]:
# uploading data folders to google colab
from google.colab import drive
drive.mount('/content/drive')

In [2]:
# reading files from drive
import os
import glob
import scipy.io
import numpy as np
import pandas as pd
import scipy.signal as signal
import matplotlib.pyplot as plt
adhd_data_path = '/content/drive/MyDrive/Dataset_EEG/ADHD_part'
control_data_path = '/content/drive/MyDrive/Dataset_EEG/Control_part'

# list all the file paths in the specified folders
adhd_data_file_paths = glob.glob(os.path.join(adhd_data_path, '*'))
control_data_file_paths = glob.glob(os.path.join(control_data_path, '*'))




In [ ]:
def data_array_createor(file_paths):
    rows = []
    # this for-loop is only to get all the rows
    for file_path in file_paths:
        content = scipy.io.loadmat(file_path)
        keys = list(content.keys())
        rows.append(content[keys[3]].shape[0])

    rows.sort()
    # since row numbers are varying, hence using the minimum row count
    min_row_count = rows[0]
    # creating empty numpy array
    data_list = []

    for file_path in file_paths:
        content = scipy.io.loadmat(file_path)
        keys = list(content.keys())
        data_list.append(content[keys[3]][0:min_row_count -1, 0:])

    data_array = np.array(data_list)
    return data_array

adhd_data_array = data_array_createor(adhd_data_file_paths)
print(adhd_data_array.shape)
control_data_array = data_array_createor(control_data_file_paths)
print(control_data_array.shape)

# slicing adhd_data_array to make it of same dimension as control_data_array
sliced_adhd_data_array = adhd_data_array[0:control_data_array.shape[0],0:control_data_array.shape[1], 0:]
print(sliced_adhd_data_array.shape)



In [ ]:
!pip install mne

In [ ]:
import mne
def eeg_signal_plot(eeg_data, channel_name_list):

    individuals, samples, channels = eeg_data.shape
    sampling_rate = 128

    # Define channel names and types
    channel_names = channel_name_list
    channel_types = ['eeg'] * channels

    for i in range(individuals):
    # Select data for the current patient
        individual_data = eeg_data[i]

    # Create MNE Info object
        info = mne.create_info(ch_names=channel_names, sfreq=sampling_rate, ch_types=channel_types)

    # Set a standard 10-20 montage
        montage = mne.channels.make_standard_montage('standard_1020')
        info.set_montage(montage)

    # Create Raw object
        # As MNE expects (channels, samples)
        raw_data = mne.io.RawArray(individual_data.T, info)
        raw_data.plot(scalings='auto')


channel_seq = ['Fp1','Fp2','F3','F4','C3','C4','P3','P4','O1','O2','F7','F8','T7','T8','P7','P8','Fz','Cz','Pz']
eeg_signal_plot(sliced_adhd_data_array,channel_seq)
eeg_signal_plot(control_data_array,channel_seq)

In [ ]:
# FFT for calculating amplitude and frequency and plot them
from scipy.fft import fft, fftfreq
def apply_fft(eeg_data):
    # calculates creates frequency bins and calculates amplitudes based on the positive cycles only
    # Sampling rate (Hz)
    fs = 128
    # Apply FFT along the sample axis (axis=1)
    eeg_data_fft = fft(eeg_data, axis=1)
    # Get the corresponding frequency bins
    freqs = fftfreq(eeg_data.shape[1], 1 / fs)
    # Only keep the positive frequencies for and FFT coefficients corresponding to positive frequencies of ADHD
    positive_freqs = freqs[:eeg_data.shape[1] // 2]
    eeg_data_fft_positive = eeg_data_fft[:, :eeg_data.shape[1] // 2, :]
    return positive_freqs, eeg_data_fft_positive

positive_freqs_adhd, eeg_data_fft_positive_adhd = apply_fft(sliced_adhd_data_array)
print('Frequency bins',positive_freqs_adhd)
print('EEG data for positive cycles',eeg_data_fft_positive_adhd)
positive_freqs_control, eeg_data_fft_positive_control = apply_fft(control_data_array)


In [ ]:

plt.figure()
plt.title('Control Data analysis in Frequency Domain')
plt.xlabel('Frequency')
plt.ylabel('Power')
for i in range(eeg_data_fft_positive_control.shape[0]):
    for j in range(eeg_data_fft_positive_control.shape[2]):
        plt.plot(positive_freqs_control, np.abs(eeg_data_fft_positive_control[i,:,j]))
plt.show()

In [ ]:

plt.figure()
plt.title('ADHD Data analysis in Frequency Domain')
plt.xlabel('Frequency')
plt.ylabel('Power')
for i in range(eeg_data_fft_positive_control.shape[0]):
    for j in range(eeg_data_fft_positive_control.shape[2]):
        plt.plot(positive_freqs_adhd, np.abs(eeg_data_fft_positive_adhd[i,:,j]))
plt.show()


In [ ]:
# implementing notch filter for removal of power line noise

def power_line_noise_removal(eeg_data):
    patients, samples, channels = eeg_data.shape
    sampling_rate = 128

    # Define channel names and types
    channel_names = ['Fp1','Fp2','F3','F4','C3','C4','P3','P4','O1','O2','F7','F8','T7','T8','P7','P8','Fz','Cz','Pz']
    channel_types = ['eeg'] * channels

    corrected_data_list = []

    for i in range(patients):
    # Select data for the current patient
        patient_data = eeg_data[i]

    # Create MNE Info object
        info = mne.create_info(ch_names=channel_names, sfreq=sampling_rate, ch_types=channel_types)

    # Create Raw object
        raw_data = mne.io.RawArray(patient_data.T, info)  # As MNE expects (channels, samples)
    # Adding a standard montage
        montage = mne.channels.make_standard_montage('standard_1020')
        raw_data.set_montage(montage)
    # Apply the 50 Hz notch filter
        raw_data.notch_filter(freqs=50, picks='eeg', filter_length='auto', phase='zero')

        corrected_data_list.append(raw_data.get_data().T)  # Shape back to (n_samples, n_channels)
    return np.array(corrected_data_list)

corrected_adhd_data = power_line_noise_removal(sliced_adhd_data_array)
corrected_control_data = power_line_noise_removal(control_data_array)

In [ ]:

positive_freqs_adhd1, eeg_data_fft_positive_adhd1 = apply_fft(corrected_adhd_data)
positive_freqs_control1, eeg_data_fft_positive_control1 = apply_fft(corrected_control_data)

plt.figure()
plt.title('Control Data analysis in Frequency Domain')
plt.xlabel('Frequency')
plt.ylabel('Amplitude')
for i in range(eeg_data_fft_positive_control1.shape[0]):
    for j in range(eeg_data_fft_positive_control1.shape[2]):
        plt.plot(positive_freqs_control1, np.abs(eeg_data_fft_positive_control1[i,:,j]))
plt.show()
plt.figure()
plt.title('ADHD Data analysis in Frequency Domain')
plt.xlabel('Frequency')
plt.ylabel('Amplitude')
for i in range(eeg_data_fft_positive_adhd1.shape[0]):
    for j in range(eeg_data_fft_positive_adhd1.shape[2]):
        plt.plot(positive_freqs_adhd1, np.abs(eeg_data_fft_positive_adhd1[i,:,j]))
plt.show()

In [ ]:
# Denoising the EEG signal using bandpass signal. It Removes baseline wander as well noise at slow frequency
def bandpass_filter(data, low_cutoff, high_cutoff, sample_rate, order=8):
   # bandpass_filter passes bands between low_cuffoff frequency and  high_cutoff frequency.

# normalising the cutoff frequencies
    nyquist = 0.5 * sample_rate  # Nyquist frequency
    low = low_cutoff / nyquist
    high = high_cutoff / nyquist
    # it returns b and a for butterworth (IIR type - infinite impulse response) filter
    b, a = signal.butter(order, [low, high], btype='bandpass', analog=False, output='ba')
    # Apply the bandpass filter using filtfilt
    filtered_signal = signal.filtfilt(b, a, data, padlen=18)  # Zero-phase filtering
    return filtered_signal

sample_rate = 128
low_cutoff = 0.5  # Low cutoff frequency in Hz
high_cutoff = 40  # High cutoff frequency in Hz
order = 4
filtered_ADHD_signal = bandpass_filter(corrected_adhd_data, low_cutoff, high_cutoff, sample_rate, order)
print(filtered_ADHD_signal.shape)
filtered_Control_signal = bandpass_filter(corrected_control_data, low_cutoff, high_cutoff, sample_rate, order)
print(filtered_ADHD_signal.shape)




In [ ]:
# Plot the original and Bandpass filtered ADHD signals
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 6))
plt.subplot(2, 1, 1)
for i in range(corrected_adhd_data.shape[0]):
    plt.plot(corrected_adhd_data[i])
plt.title('Original ADHD Signal')
plt.xlabel('Time in milli seconds')
plt.grid()

plt.subplot(2, 1, 2)
for i in range(filtered_ADHD_signal.shape[0]):
    plt.plot(filtered_ADHD_signal[i])
plt.title('Filtered ADHD Signal by Bandpass Filter')
plt.xlabel('Time in milli seconds')
plt.grid()

plt.tight_layout()
plt.show()

In [ ]:
# Plot the Original and Bandpass filtered Control signals
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 6))
plt.subplot(2, 1, 1)
for i in range(corrected_control_data.shape[0]):
    plt.plot(corrected_control_data[i])
plt.title('Original Control Signal')
plt.xlabel('Time in milli seconds')
plt.grid()

plt.subplot(2, 1, 2)
for i in range(filtered_Control_signal.shape[0]):
    plt.plot(filtered_Control_signal[i])
plt.title('Filtered Control Signal by Bandpass Filter')
plt.xlabel('Time in milli seconds')
plt.grid()

plt.tight_layout()
plt.show()

In [ ]:
# applying ICA from mne library to remove further artefacts
import mne
from mne.preprocessing import ICA
def implement_ICA(eeg_data):
    candidates, samples, channels = eeg_data.shape
    sampling_rate = 128

    # Define channel names and types
    channel_names = ['Fp1','Fp2','F3','F4','C3','C4','P3','P4','O1','O2','F7','F8','T7','T8','P7','P8','Fz','Cz','Pz']
    channel_types = ['eeg'] * channels

    ica_results = []
    corrected_data_list = []

    for i in range(candidates):
    # Select data for the current patient
        patient_data = eeg_data[i]

    # Create MNE Info object
        info = mne.create_info(ch_names=channel_names, sfreq=sampling_rate, ch_types=channel_types)

    # Set a standard 10-20 montage
        montage = mne.channels.make_standard_montage('standard_1020')
        info.set_montage(montage)

    # Create Raw object
        raw_data = mne.io.RawArray(patient_data.T, info)  # As MNE expects (channels, samples)

    # Apply ICA
        ica = ICA(n_components=15, random_state=97, max_iter='auto') # to reduce overfitting , n_components =15
        ica.fit(raw_data)

    # Plot ICA components (optional, to inspect visually)
        ica.plot_components(title=f"Patient {i+1} ICA components")

    # finding EOG artifacts
        eog_indices, eog_scores = ica.find_bads_eog(raw_data, ch_name=['Fp1', 'Fp2','F7', 'F8'])
        ica.exclude = eog_indices  # exclude components for EOG artifacts

    # Store the ICA object for later use
        ica_results.append(ica)

    # Apply the ICA solution to the data to remove the artifacts
        raw_corrected = ica.apply(raw_data.copy())

        corrected_data_list.append(raw_corrected.get_data().T)  # Shape back to (n_samples, n_channels)
    return ica_results, np.array(corrected_data_list)

ica_control, corrected_control_data = implement_ICA(filtered_Control_signal)
ica_adhd, corrected_adhd_data = implement_ICA(filtered_ADHD_signal)

In [ ]:
corrected_control_data.shape

In [ ]:
# removal of muscle artefacts

def ICA_for_muscle_artefact_removal(eeg_data):
    candidates, samples, channels = eeg_data.shape
    sampling_rate = 128

    # Define channel names and types
    channel_names = ['Fp1','Fp2','F3','F4','C3','C4','P3','P4','O1','O2','F7','F8','T7','T8','P7','P8','Fz','Cz','Pz']
    channel_types = ['eeg'] * channels

    ica_results = []
    corrected_data_list = []

    for i in range(candidates):
    # Select data for the current patient
        candidate_data = eeg_data[i]

    # Create MNE Info object
        info = mne.create_info(ch_names=channel_names, sfreq=sampling_rate, ch_types=channel_types)


    # Create Raw object
        raw_data = mne.io.RawArray(candidate_data.T, info)  # As MNE expects (channels, samples)
    # Adding a standard montage
        montage = mne.channels.make_standard_montage('standard_1020')
        raw_data.set_montage(montage)
        raw_data.filter(l_freq=1.0, h_freq=None) # highpass

    # Apply ICA
        ica = ICA(n_components=15, random_state=97, max_iter='auto') # to reduce overfitting , n_components =15
        ica.fit(raw_data)

        muscle_index, scores = ica.find_bads_muscle(raw_data)
        ica.exclude = muscle_index  # exclude components for muscle artifacts
        ica.plot_scores(scores, exclude=muscle_index)
        print(f"Automatically found muscle artifact ICA components: {muscle_index}")

    # Store the ICA object for later use
        ica_results.append(ica)

    # Apply the ICA solution to the data to remove the artifacts
        raw_corrected = ica.apply(raw_data.copy())

        corrected_data_list.append(raw_corrected.get_data().T)  # Shape back to (n_samples, n_channels)
    return ica_results, np.array(corrected_data_list)

ica_control1, corrected_control_data1 = ICA_for_muscle_artefact_removal(corrected_control_data)
ica_adhd1, corrected_adhd_data1 = ICA_for_muscle_artefact_removal(corrected_adhd_data)

In [ ]:
positive_freqs_adhd2, eeg_data_fft_positive_adhd2 = apply_fft(corrected_control_data1)
positive_freqs_control2, eeg_data_fft_positive_control2 = apply_fft(corrected_adhd_data1)

plt.figure()
plt.title('Control Data analysis in Frequency Domain')
plt.xlabel('Frequency')
plt.ylabel('Power')
for i in range(eeg_data_fft_positive_control2.shape[0]):
    for j in range(eeg_data_fft_positive_control2.shape[2]):
        plt.plot(positive_freqs_control2, np.abs(eeg_data_fft_positive_control2[i,:,j]))
plt.show()
plt.figure()
plt.title('ADHD Data analysis in Frequency Domain')
plt.xlabel('Frequency')
plt.ylabel('Power')
for i in range(eeg_data_fft_positive_adhd2.shape[0]):
    for j in range(eeg_data_fft_positive_adhd2.shape[2]):
        plt.plot(positive_freqs_adhd2, np.abs(eeg_data_fft_positive_adhd2[i,:,j]))
plt.show()

In [ ]:
# creating label for ADHD data
adhd_label = np.ones(corrected_adhd_data1.shape[0])
print(adhd_label)
#creating control label
control_label = np.zeros(corrected_control_data1.shape[0])
print(control_label.shape)
#combining the label arrays
combined_label_array = np.concatenate((adhd_label, control_label))
print(combined_label_array.shape)
print(combined_label_array)





In [ ]:
corrected_adhd_data1.shape

In [ ]:
# combining ADHD and control data arrays
combined_data_array = np.concatenate((corrected_adhd_data1, corrected_control_data1), axis=0)
print(combined_data_array.shape)






In [ ]:
#plotting eeg signals after preprocessing
eeg_signal_plot(corrected_adhd_data1,channel_seq)
eeg_signal_plot(corrected_control_data1,channel_seq)

In [ ]:
!pip install PyWavelets

In [ ]:
import pywt
def wavelet_artefact_removal(eeg_data, wavelet, level, threshold):
    """
    Apply discrete wavelet transform to EEG data.

    Returns:
         EEG data without artefacts
    """
# Process each channel for each patient
    for candidate in range(eeg_data.shape[0]):
        for channel in range(eeg_data.shape[2]):
            # Extract the signal for the current patient and channel
            signal = eeg_data[candidate, :, channel]

            # Perform wavelet decomposition
            coeffecients = pywt.wavedec(signal, wavelet, mode='symmetric',level=level)

            # Apply soft thresholding to the detail coefficients (artifact removal)
            coeffs_thresholded = [coeffecients[0]]  # Approximation coefficients
            for i in range(1, len(coeffecients)):
                coeffs_thresholded.append(pywt.threshold(coeffecients[i], threshold * np.max(coeffecients[i]), mode='hard'))

            # Reconstruct the signal from the thresholded coefficients
            no_artefacts_signal = pywt.waverec(coeffs_thresholded, wavelet)

            # Store the denoised signal back into the array
            eeg_data[candidate, :, channel] = no_artefacts_signal[:eeg_data.shape[1]]

    return eeg_data

# Apply wavelet denoising
wavelet = 'db4'    # Daubechies wavelet
level = 7         # Number of decomposition levels
threshold = 0.5    # Denoising threshold

no_artefact_ADHD_data = wavelet_artefact_removal(corrected_adhd_data1, wavelet, level, threshold)
no_artefact_Control_data = wavelet_artefact_removal(corrected_control_data1, wavelet=wavelet, level=level, threshold=threshold)
print(no_artefact_ADHD_data.shape)

In [ ]:
def plot_patients_and_channels_data(original_eeg_data, denoised_eeg_data, candidates, channels):
    """
    Plotting the original and denoised EEG signals for all patients and channels.
    """
    # Set up the plot
    figure, axes = plt.subplots(candidates, channels, figsize=(20, candidates * 2), sharex=True, sharey=True)
    figure.suptitle("Before vs After Denoised EEG", fontsize=16)

    # Iterates over patients and channels
    for candidate_index in range(candidates):
        for channel_index in range(channels):
            ax = axes[candidate_index, channel_index]

            # Extracts the signals for the current patient and channel
            before_wavelet= original_eeg_data[candidate_index, :, channel_index]
            after_wavelet = denoised_eeg_data[candidate_index, :, channel_index]

            # Plot original and denoised signals
            ax.plot(before_wavelet, label='Original', alpha=0.7)
            ax.plot(after_wavelet, label='Denoised', alpha=0.7, linestyle='--')

            # Remove ticks for better readability
            ax.set_xticks([])
            ax.set_yticks([])

            # Add labels to the first row and column
            if candidate_index == 0:
                ax.set_title(f"Ch {channel_index + 1}")
            if channel_index == 0:
                ax.set_ylabel(f"Patient {candidate_index + 1}", rotation=0, labelpad=20)

    # Add a single legend for the entire figure
    handles, labels = axes[0, 0].get_legend_handles_labels()  # is a built-in method in the Matplotlib library
    figure.legend(handles, labels, loc='upper center', fontsize=12, bbox_to_anchor=(0.5, 1.05), ncol=2)

    plt.tight_layout(rect=[0, 0, 0.95, 0.98])  # Adjust layout for the title and legend
    plt.show()

candidates_adhd = no_artefact_ADHD_data.shape[0]
channels = no_artefact_ADHD_data.shape[2]

candidates_control = no_artefact_Control_data.shape[0]
channels = no_artefact_Control_data.shape[2]
# Plot the results for all patients and channels
plot_patients_and_channels_data(corrected_adhd_data1, no_artefact_ADHD_data, candidates_adhd, channels)


In [ ]:
plot_patients_and_channels_data(corrected_control_data1, no_artefact_Control_data, candidates_control, channels)

In [ ]:
# time domain analysis
mean_vals = np.mean(no_artefact_ADHD_data, axis=1)
variance_vals = np.var(no_artefact_ADHD_data, axis=1)
rms_vals = np.sqrt(np.mean(no_artefact_ADHD_data**2, axis=1))
std_dev_vals = np.std(no_artefact_ADHD_data, axis=1)

channels = np.arange(1, 20)  # 19 channels

# Create subplots
figure, axs = plt.subplots(2, 2, figsize=(16, 12))
figure.suptitle("Time-Domain Metrics for All ADHD Patients")

# Plot metrics for all patients
for patient_index in range(mean_vals.shape[0]):
    # Mean values
    axs[0, 0].plot(channels, mean_vals[patient_index, :], label=f"Patient {patient_index+1}", alpha=0.5)
    axs[0, 0].set_title("Mean Values")
    axs[0, 0].set_xlabel("Channel")
    axs[0, 0].set_ylabel("Mean Amplitude")

    # Variance values
    axs[0, 1].plot(channels, variance_vals[patient_index, :], label=f"Patient {patient_index+1}", alpha=0.5)
    axs[0, 1].set_title("Variance Values")
    axs[0, 1].set_xlabel("Channel")
    axs[0, 1].set_ylabel("Variance")

    # RMS values
    axs[1, 0].plot(channels, rms_vals[patient_index, :], label=f"Patient {patient_index+1}", alpha=0.5)
    axs[1, 0].set_title("RMS Values")
    axs[1, 0].set_xlabel("Channel")
    axs[1, 0].set_ylabel("RMS Amplitude")

    # Standard deviation values
    axs[1, 1].plot(channels, std_dev_vals[patient_index, :], label=f"Patient {patient_index+1}", alpha=0.5)
    axs[1, 1].set_title("Standard Deviation Values")
    axs[1, 1].set_xlabel("Channel")
    axs[1, 1].set_ylabel("Standard Deviation")

plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

In [ ]:
# time domain analysis
mean_vals = np.mean(no_artefact_Control_data, axis=1)
variance_vals = np.var(no_artefact_Control_data, axis=1)
rms_vals = np.sqrt(np.mean(no_artefact_Control_data**2, axis=1))
std_dev_vals = np.std(no_artefact_Control_data, axis=1)

channels = np.arange(1, 20)  # 19 channels

# Create subplots
figure, axs = plt.subplots(2, 2, figsize=(16, 12))
figure.suptitle("Time-Domain Metrics for All Control Patients")

# Plot metrics for all patients
for patient_index in range(mean_vals.shape[0]):
    # Mean values
    axs[0, 0].plot(channels, mean_vals[patient_index, :], label=f"Patient {patient_index+1}", alpha=0.5)
    axs[0, 0].set_title("Mean Values")
    axs[0, 0].set_xlabel("Channel")
    axs[0, 0].set_ylabel("Mean Amplitude")

    # Variance values
    axs[0, 1].plot(channels, variance_vals[patient_index, :], label=f"Patient {patient_index+1}", alpha=0.5)
    axs[0, 1].set_title("Variance Values")
    axs[0, 1].set_xlabel("Channel")
    axs[0, 1].set_ylabel("Variance")

    # RMS values
    axs[1, 0].plot(channels, rms_vals[patient_index, :], label=f"Patient {patient_index+1}", alpha=0.5)
    axs[1, 0].set_title("RMS Values")
    axs[1, 0].set_xlabel("Channel")
    axs[1, 0].set_ylabel("RMS Amplitude")

    # Standard deviation values
    axs[1, 1].plot(channels, std_dev_vals[patient_index, :], label=f"Patient {patient_index+1}", alpha=0.5)
    axs[1, 1].set_title("Standard Deviation Values")
    axs[1, 1].set_xlabel("Channel")
    axs[1, 1].set_ylabel("Standard Deviation")

plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

In [ ]:
!pip install -U mne-connectivity
!pip install spectral_connectivity


In [ ]:
# connectivity analysis
from mne_connectivity import spectral_connectivity_epochs

def functional_connectivity_for_eeg(eeg_data, sfreq):
    """
    Compute functional connectivity metrics (correlation and coherence) for all patients and frequency bands.
    Returns 'result' a dict
    """
    # Frequency bands
    bands = {
        "Delta": (0.5, 4),
        "Theta": (4, 8),
        "Alpha": (8, 13),
        "Beta": (13, 30),
        "Gamma": (30, 50)
    }

    candidates, samples, channels = eeg_data.shape
    result = {}

    # Define channel names and types
    channel_names = [
        'Fp1', 'Fp2', 'F3', 'F4', 'C3', 'C4', 'P3', 'P4', 'O1', 'O2',
        'F7', 'F8', 'T7', 'T8', 'P7', 'P8', 'Fz', 'Cz', 'Pz'
    ]
    channel_types = ['eeg'] * channels

    for candidate in range(candidates):
        candidate_results = {}
        eeg_individual_data = eeg_data[candidate].T  # Shape: (channels, samples)

        # Create an MNE Raw object
        info = mne.create_info(ch_names=channel_names, sfreq=sfreq, ch_types=channel_types)
        raw_data = mne.io.RawArray(eeg_individual_data, info)

        # Compute Correlation Matrix
        correlation_matrix = np.corrcoef(eeg_individual_data)
        candidate_results["Correlation"] = correlation_matrix

        # Compute Coherence for each frequency band
        band_results = {}
        for band, (fmin, fmax) in bands.items():
            # Spectral connectivity calculation
            con = spectral_connectivity_epochs(
                np.expand_dims(raw_data.get_data(), axis=0),  # Add an epoch dimension
                sfreq=sfreq,
                method="coh",
                mode="multitaper",
                fmin=fmin,
                fmax=fmax,
                faverage=True,
                verbose=False
            )
            coherence_matrix = con.get_data(output="dense")[:, :, 0]  # Extract coherence values
            band_results[band] = coherence_matrix

        candidate_results["Coherence"] = band_results
        result[f"Patient_{candidate}"] = candidate_results

    return result



sfreq = 128
# Computing functional connectivity
results_adhd = functional_connectivity_for_eeg(no_artefact_ADHD_data, sfreq=sfreq)
results_control = functional_connectivity_for_eeg(no_artefact_Control_data, sfreq=sfreq)



In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
def plot_coherence_matrices(results, bands):
    """
    Plots the coherence matrices for each patient and each frequency band.
    Parameters:
    - results: dict
        The dictionary contains the computed coherence matrices for each patient and frequency band.
    - bands: dict
        The dictionary containing the frequency bands.
    """
    # Iterate over all the patients
    for patient_id, patient_results in results.items():
        band_results = patient_results["Coherence"]

        # plot for each frequency band
        figure, axes = plt.subplots(1, len(bands), figsize=(15, 5))

        # If only one band, axes will be a single object, so make sure it's a list
        if len(bands) == 1:
            axes = [axes]

        # Loop through each frequency band
        for i, (band, _) in enumerate(bands.items()):
            # Get the coherence matrix for the current band
            coherence_matrix = band_results[band]

            # Plot the coherence matrix as a heatmap
            sns.heatmap(coherence_matrix, ax=axes[i], cmap="coolwarm", annot=False, xticklabels=False, yticklabels=False)
            axes[i].set_title(f"{band} Band")
            axes[i].set_xlabel('Channels')
            axes[i].set_ylabel('Channels')

        # Set a title for the whole figure
        figure.suptitle(f"Coherence Matrices for Patient {patient_id}", fontsize=16)
        plt.tight_layout(rect=[0, 0.03, 1, 0.95])  # This is to adjust layout to avoid title overlap
        plt.show()


#results_adhd is the dictionary containing the coherence results for all patients
bands = {
        "Delta": (0.5, 4),
        "Theta": (4, 8),
        "Alpha": (8, 13),
        "Beta":  (13, 30),
        "Gamma": (30, 50)
    }
plot_coherence_matrices(results_adhd, bands)
plot_coherence_matrices(results_control, bands)

In [ ]:
# PSD Analysis
from mne.time_frequency import psd_array_welch

def spectral_density_analysis(eeg_data, sfreq, plot=True):
    """
    This function is for spectral density analysis and plot the results using MNE.
    This function returns psds an ndarray
    Power spectral density values of shape (channels, freqs).
    """
    candidates, samples, channels = eeg_data.shape
    fmin=0.1
    fmax=50.0

    # Initialize MNE Info object
       # Define channel names and types
    channel_names = [
        'Fp1', 'Fp2', 'F3', 'F4', 'C3', 'C4', 'P3', 'P4', 'O1', 'O2',
        'F7', 'F8', 'T7', 'T8', 'P7', 'P8', 'Fz', 'Cz', 'Pz']
    channel_types = ['eeg'] * channels
    info = mne.create_info(ch_names=channel_names, sfreq=sfreq, ch_types=channel_types)

    # Create MNE RawArray for each trial and compute PSD
    psds_list = []
    for trial in eeg_data:
        raw_data = mne.io.RawArray(trial.T, info, verbose=False)
        trial_psd, freqs = psd_array_welch(trial.T, sfreq=sfreq, fmin=fmin, fmax=fmax, n_fft=1024)
        psds_list.append(trial_psd)

    # Average PSD across trials
    psds_list = np.mean(psds_list, axis=0)  # Shape: (channels, freqs)

    if plot:
        plt.figure(figsize=(10, 6))
        for ch_index, ch_name in enumerate(channel_names):
            plt.plot(freqs, psds_list[ch_index], label=ch_name)
        plt.xlabel('Frequency (Hz)')
        plt.ylabel('Power Spectral Density (dB)')
        plt.title('Average Power Spectral Density Across Trials')
        plt.legend(loc='upper right', bbox_to_anchor=(1.15, 1))
        plt.grid()
        plt.tight_layout()
        plt.show()

    return psds_list, freqs


sfreq = 128
psds_adhd, freqs_adhd = spectral_density_analysis(no_artefact_ADHD_data, sfreq)
#psds_control, freqs_control = spectral_density_analysis(no_artefact_Control_data, sfreq)

In [ ]:
psds_control, freqs_control = spectral_density_analysis(no_artefact_Control_data, sfreq)

In [ ]:
# EEG data is loaded as eeg_data with shape (n_trials, n_samples, n_channels)
# sfreq is the sampling frequency
def compute_psd_features(eeg_data, sfreq, fmin=0.1, fmax=50.0):
    n_trials, n_samples, n_channels = eeg_data.shape
    fmin= 0.1
    fmax= 50.0

    # Define channel names and types
    channel_names = [
        'Fp1', 'Fp2', 'F3', 'F4', 'C3', 'C4', 'P3', 'P4', 'O1', 'O2',
        'F7', 'F8', 'T7', 'T8', 'P7', 'P8', 'Fz', 'Cz', 'Pz']
    channel_types = ['eeg'] * n_channels
    # Initialize MNE Info object
    info = mne.create_info(ch_names=channel_names, sfreq=sfreq, ch_types=channel_types)

    # Creating MNE RawArray for each trial and computing PSD
    psds_list = []
    for trial in eeg_data:
        raw_data = mne.io.RawArray(trial.T, info, verbose=False)
        trial_psds, freqs = psd_array_welch(trial.T, sfreq=sfreq, fmin=fmin, fmax=fmax, n_fft=1024)
        psds_list.append(trial_psds)
    return np.array(psds_list), freqs


sfreq = 128
psd_features_control, freqs_ontrol = compute_psd_features(no_artefact_Control_data, sfreq)
psd_features_control.shape
psd_features_adhd, freqs_adhd = compute_psd_features(no_artefact_ADHD_data, sfreq)
psd_features_adhd.shape

In [ ]:

combined_data_array1 = np.concatenate((psd_features_adhd, psd_features_control), axis=0)
print(combined_data_array1.shape)

In [ ]:
# creating label for ADHD data
adhd_label = np.ones(corrected_adhd_data1.shape[0])
print(adhd_label)
#creating control label
control_label = np.zeros(corrected_control_data1.shape[0])
print(control_label.shape)
#combining the label arrays
combined_label_array = np.concatenate((adhd_label, control_label))
print(combined_label_array.shape)
print(combined_label_array)

In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC


# Split data into train and test
xtrain, xtest, ytrain, ytest = train_test_split(combined_data_array1, combined_label_array, test_size=0.2, random_state=105)

# One-hot encode the labels for classification
ytrain = to_categorical(ytrain, num_classes=2)
ytest = to_categorical(ytest, num_classes=2)

xtrain.shape


In [ ]:
ytrain.shape

In [ ]:
xtrain_flattened = xtrain.reshape(xtrain.shape[0], -1)
xtest_flattened = xtest.reshape(xtest.shape[0], -1)

# For SVM, we need a single label per sample, so we use one of the columns
ytrain_flattened = ytrain[:, 0]
ytest_flattened = ytest[:, 0]

training_errors = []
validation_errors = []


# Initialize and train the SVM model
model = SVC(kernel='linear', C=1, random_state=24)
model.fit(xtrain_flattened, ytrain_flattened)
# predictions
ytrain_pred = model.predict(xtrain_flattened)
ypred = model.predict(xtest_flattened)
# Evaluate the model
training_accuracy = accuracy_score(ytrain_flattened, ytrain_pred)
accuracy = accuracy_score(ytest_flattened, ypred)
print(f"Accuracy: {accuracy:.2f}")
# Calculate errors (1 - accuracy)
training_error = 1 - training_accuracy
validation_error = 1 - accuracy
print(f"Validation Error: {validation_error:.2f}")



In [ ]:
# Data for the bar plot
types_of_error = ['Training Error', 'Validation Error']
errors = [training_error, validation_error]

# Create a bar plot
plt.figure(figsize=(8, 6))
plt.bar(types_of_error, errors, color=['blue', 'red'], alpha=0.7)

# Add labels and title
plt.title("Training vs Validation Error for SVM", fontsize=14)
plt.ylabel("Error", fontsize=12)
plt.ylim(0, 1)  # Assuming errors are in the range [0, 1]

# Show the plot
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

In [ ]:
# Data for the bar plot
accuracies_type = ['Training Accuracy', 'Validation Accuracy']
accuracy = [training_accuracy, accuracy]

# Create a bar plot
plt.figure(figsize=(8, 6))
plt.bar(accuracies_type, accuracy, color=['blue', 'red'], alpha=0.7)

plt.title("Training vs Validation Accuracy of SVM", fontsize=14)
plt.ylabel("Accuracy", fontsize=12)
plt.ylim(0, 1)

# Show the plot
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler

# Scaling the features
scaler = StandardScaler()
xtrain_flattened_scaled = scaler.fit_transform(xtrain_flattened)
xtest_flattened_scaled = scaler.transform(xtest_flattened)

#hyper parameter tuning
params = {
    'C': [0.5, 1, 2, 3, 4],
    'kernel': ['linear', 'rbf'],
    'gamma': ['scale', 'auto'],
}

# Initialize GridSearchCV with 5-fold cross-validation
gscv = GridSearchCV(SVC(), params, cv=5, n_jobs=-1)
gscv.fit(xtrain_flattened_scaled, ytrain_flattened)

# Get the best parameters
print(f"Best parameters: {gscv.best_params_}")
print(f"Best cross-validation score: {gscv.best_score_:.2f}")

In [ ]:

# Initialising and train the SVM model with cross-validation
model_gscv = SVC(kernel='rbf', C=1, gamma='auto')
model_gscv.fit(xtrain_flattened_scaled, ytrain_flattened)

# predictions
ytrain_pred_gscv = model_gscv.predict(xtrain_flattened_scaled)
ypred_gscv = model_gscv.predict(xtest_flattened_scaled)

# Evaluation of the model
training_accuracy_gscv = accuracy_score(ytrain_flattened, ytrain_pred_gscv)
val_accuracy_gscv = accuracy_score(ytest_flattened, ypred_gscv)
print(f"Accuracy: {val_accuracy_gscv:.2f}")
training_error_gscv = 1 - training_accuracy_gscv
validation_error_gscv = 1 - val_accuracy_gscv
print(f"Training Error: {training_error_gscv:.2f}")
print(f"Validation Error: {validation_error_gscv:.2f}")


In [ ]:
# Data for the bar plot
types_of_error_gscv = ['Training Error', 'Validation Error']
errors_gscv = [training_error_gscv, validation_error_gscv]

# Create a bar plot
plt.figure(figsize=(8, 6))
plt.bar(types_of_error_gscv, errors_gscv, color=['blue', 'red'], alpha=0.7)

plt.title("Training vs Validation Error for SVM after Hyper-Parameter Tuning", fontsize=14)
plt.ylabel("Error", fontsize=12)
plt.ylim(0, 1)

# Show the plot
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

In [ ]:
# Data for the bar plot
accuracies_type_gscv = ['Training Accuracy', 'Validation Accuracy']
accuracy_gscv = [training_accuracy_gscv, val_accuracy_gscv]

# Create a bar plot
plt.figure(figsize=(8, 6))
plt.bar(accuracies_type_gscv, accuracy_gscv, color=['blue', 'red'], alpha=0.7)

plt.title("Training vs Validation Accuracy of SVM after Hyper-Parameter Tuning", fontsize=14)
plt.ylabel("Accuracy", fontsize=12)
plt.ylim(0, 1)

# Show the plot
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

In [ ]:
from xgboost import XGBClassifier
# Initialize and train the XGBoost model
model_xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=102)
model_xgb.fit(xtrain_flattened, ytrain_flattened)

# Make predictions
ytrain_pred_xgb = model_xgb.predict(xtrain_flattened)
ypred_xgb = model_xgb.predict(xtest_flattened)

# Evaluate the model
training_accuracy_xgb = accuracy_score(ytrain_flattened, ytrain_pred_xgb)
accuracy_xgb = accuracy_score(ytest_flattened, ypred_xgb)
print(f"Accuracy: {accuracy_xgb:.2f}")

# Calculate errors (1 - accuracy)
training_error_xgb = 1 - training_accuracy_xgb
validation_error_xgb = 1 - accuracy_xgb

# Print errors
print(f"Training Error: {training_error_xgb:.2f}")
print(f"Validation Error: {validation_error_xgb:.2f}")

In [ ]:
# Data for the bar plot
types_of_error_xgb = ['Training Error', 'Validation Error']
errors_xgb = [training_error_xgb, validation_error_xgb]

# Create a bar plot
plt.figure(figsize=(8, 6))
plt.bar(types_of_error_xgb, errors_xgb, color=['blue', 'red'], alpha=0.7)

# Add labels and title
plt.title("Training vs Validation Error for XG Boost", fontsize=14)
plt.ylabel("Error", fontsize=12)
plt.ylim(0, 1)  # Assuming errors are in the range [0, 1]

# Show the plot
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

In [ ]:
# Data for the bar plot
accuracies_type_xgb = ['Training Accuracy', 'Validation Accuracy']
val_accuracy_xgb = [training_accuracy_xgb, accuracy_xgb]

# Create a bar plot
plt.figure(figsize=(8, 6))
plt.bar(accuracies_type_xgb, val_accuracy_xgb, color=['blue', 'red'], alpha=0.7)

# Add labels and title
plt.title("Training vs Validation Accuracy of XG Boost", fontsize=14)
plt.ylabel("Accuracy", fontsize=12)
plt.ylim(0, 1)  # Accuracies are in the range [0, 1]

# Show the plot
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

In [ ]:
# hyperparameter tuning for XGBoost
param_grid_xgb = {
    'n_estimators': [25, 50, 75],
    'max_depth': [3, 5],
    'learning_rate': [0.1, 0.2],
}

# Initialising GridSearchCV with 5-fold cross-validation
model_xgb = XGBClassifier(random_state=24, eval_metric='logloss')
gscv_xgb = GridSearchCV(estimator=model_xgb, param_grid=param_grid_xgb, cv=5, scoring='accuracy', n_jobs=-1)

# Fit the model to the training data
gscv_xgb.fit(xtrain_flattened_scaled, ytrain_flattened)

# Get the best parameters and the best model
best_params = gscv_xgb.best_params_




In [ ]:
print(best_params)

In [ ]:
#Initialize and train the XGBoost model
model_xgb_best = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=102, max_depth=5, learning_rate=0.1,n_estimators=25)
model_xgb_best.fit(xtrain_flattened, ytrain_flattened)

# Make predictions
ytrain_pred_xgb_best = model_xgb_best.predict(xtrain_flattened)
ypred_xgb_best = model_xgb_best.predict(xtest_flattened)

# Evaluate the model
training_accuracy_xgb_best = accuracy_score(ytrain_flattened, ytrain_pred_xgb_best)
accuracy_xgb_best = accuracy_score(ytest_flattened, ypred_xgb_best)
print(f"Accuracy: {accuracy_xgb_best:.2f}")

# Calculate errors (1 - accuracy)
training_error_xgb_best = 1 - training_accuracy_xgb_best
validation_error_xgb_best = 1 - accuracy_xgb_best

# Print errors
print(f"Training Error: {training_error_xgb_best:.2f}")
print(f"Validation Error: {validation_error_xgb_best:.2f}")

In [ ]:
# Data for the bar plot
accuracies_type_xgb_best = ['Training Accuracy', 'Validation Accuracy']
val_accuracy_xgb_best = [training_accuracy_xgb_best, accuracy_xgb_best]

# Create a bar plot
plt.figure(figsize=(8, 6))
plt.bar(accuracies_type_xgb_best, val_accuracy_xgb_best, color=['blue', 'red'], alpha=0.7)

# Add labels and title
plt.title("Training vs Validation Accuracy of XG Boost after Tuning", fontsize=14)
plt.ylabel("Accuracy", fontsize=12)
plt.ylim(0, 1)  # as Accuracies are in the range [0, 1]

# Show the plot
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Dropout, BatchNormalization
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical


# Reshaping PSD features for CNN (trials, channels, freqs, 1)
combined_data_array1 = combined_data_array1.reshape(combined_data_array1.shape[0], combined_data_array1.shape[1], combined_data_array1.shape[2], 1)

# Split data into train and test
xtrain, xtest, ytrain, ytest = train_test_split(combined_data_array1, combined_label_array, test_size=0.2, random_state=105)

# One-hot encode the labels for classification
ytrain = to_categorical(ytrain, num_classes=2)
ytest = to_categorical(ytest, num_classes=2)

# Create CNN model
model = Sequential()

# First Conv2D layer
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', padding='valid', input_shape=(combined_data_array1.shape[1], combined_data_array1.shape[2], 1)))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='valid'))

# Second Conv2D layer
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu', padding='valid'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='valid'))

# Third Conv2D layer with strided convolution
model.add(Conv2D(256, kernel_size=(3, 3), activation='relu', strides=2, padding='valid'))

# Flatten the output of the last Conv2D layer
model.add(Flatten())

# Fully connected layers
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(2, activation='softmax'))  # Output layer for binary classification

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(xtrain, ytrain, validation_data=(xtest, ytest), epochs=10, batch_size=32)

# Print the model summary
model.summary()

In [ ]:

plt.figure()
plt.plot(range(10), history.history['loss'], label='loss')
plt.plot(range(10), history.history['val_loss'], label='val_loss')
plt.legend()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
plt.figure()
plt.plot(range(10), history.history['accuracy'], label='accuracy')
plt.plot(range(10), history.history['val_accuracy'], label='val_accuracy')
plt.legend()
plt.show()

In [ ]:
# after tuning the model
model = Sequential()

# First Conv2D layer
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', padding='valid', input_shape=(combined_data_array1.shape[1], combined_data_array1.shape[2], 1)))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='valid'))

# Second Conv2D layer
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu', padding='valid'))
model.add(BatchNormalization())
model.add(Dropout(0.4))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='valid'))

# Third Conv2D layer with strided convolution
model.add(Conv2D(256, kernel_size=(3, 3), activation='relu', strides=2, padding='valid'))
model.add(BatchNormalization())
model.add(Dropout(0.4))

# Flatten last Conv2D layer
model.add(Flatten())

# Fully connected layers
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(2, activation='softmax'))  # Output layer for binary classification

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(xtrain, ytrain, validation_data=(xtest, ytest), epochs=10, batch_size=32)

# Print the model summary
model.summary()

In [ ]:
import matplotlib.pyplot as plt
plt.figure()
plt.plot(range(10), history.history['loss'], label='loss')
plt.plot(range(10), history.history['val_loss'], label='val_loss')
plt.legend()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
plt.figure()
plt.plot(range(10), history.history['accuracy'], label='accuracy')
plt.plot(range(10), history.history['val_accuracy'], label='val_accuracy')
plt.legend()
plt.show()